<a href="https://colab.research.google.com/github/tarrantcarter/Data-Science-Python-Basics/blob/master/Most_Popular_Posts_and_Commenting_Times_on_Hacker_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Most Popular Posts and Commenting Times on Hacker News

Our goal for this project is to determine which type of posts are the most popular for users of Hacker News (hackernews.com). We would also like to know what time period for posting an article that receives the most engagment from users. This will allow us to create more timely and targeted posts in the future. 

# Opening and Exploring the Data

We will begin by examing a free hacker news dataset found [here.](https://www.kaggle.com/hacker-news/hacker-news-posts)
Let's begin by opening the data set.

In [ ]:
from csv import reader

### The Hacker News data set ###
opened_file = open('hacker_news.csv')
read_file = reader(opened_file)
hn = list(read_file)
hn_header = hn[0]
hn = hn[1:]

Next we will write a function called explore_data() that will allow us to display the first 5 rows of the dataset. Also, we will add to our function the ability to determine the number or rows and columns from each data set.

In [ ]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))  

Let's take a look at the hacker news dataset. 

In [ ]:
print(hn_header)
print("\n")
first_five_rows = explore_data(hn,0,4,True)
print(first_five_rows)

['id', 'title', 'url', 'num_points', 'num_comments', 'author', 'created_at']


['12224879', 'Interactive Dynamic Video', 'http://www.interactivedynamicvideo.com/', '386', '52', 'ne0phyte', '8/4/2016 11:52']


['10975351', 'How to Use Open Source and Shut the Fuck Up at the Same Time', 'http://hueniverse.com/2016/01/26/how-to-use-open-source-and-shut-the-fuck-up-at-the-same-time/', '39', '10', 'josep2', '1/26/2016 19:30']


['11964716', "Florida DJs May Face Felony for April Fools' Water Joke", 'http://www.thewire.com/entertainment/2013/04/florida-djs-april-fools-water-joke/63798/', '2', '1', 'vezycash', '6/23/2016 22:20']


['11919867', 'Technology ventures: From Idea to Enterprise', 'https://www.amazon.com/Technology-Ventures-Enterprise-Thomas-Byers/dp/0073523429', '3', '1', 'hswarna', '6/17/2016 0:01']


Number of rows: 20100
Number of columns: 7
None


We have 20,100 articles in the data set and the columns that appear relevent for our analysis are:'title', 'num_comments', and 'created_at'. The column titles seem mostly self-explanatory. Below is a detailed explaination of each column title which was taken from the [description tab of the project.](https://www.kaggle.com/hacker-news/hacker-news-posts)

The 7 column discriptions include:

- title: title of the post (self-explanatory)

- url: the url of the item being linked to

- num_points: the number of upvotes the post received

- num_comments: the number of comments the post received

- author: the name of the account that made the post

- created_at: the date and time the post was made (the time zone is Eastern Time in the US)

# Isolating Ask HN and Show HN posts

Before we can determine which is more popular, we need to isolate Ask HN and Show HN posts. To to this will loop through our hn dataset and put the Ask HN, Show HN, and all other articles in seperate lists. We will use the .lower() and .startwith() method to help us parse the strings in 'title' column.

In [ ]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith("ask hn"):
        ask_posts.append(row)
    elif title.startswith("show hn"):
        show_posts.append(row)
    else:
        other_posts.append(row)
        
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


As we can see there are many more Ask HN posts (1744) than Show HN posts (1162).

In the previous screen we seperated the Ask HN posts and Show HN posts into two list of lists. Below are the first five rows from the ask_posts list of lists:

In [ ]:
ask_first_five = explore_data(ask_posts,0,4)
print(ask_first_five)

['12296411', 'Ask HN: How to improve my personal website?', '', '2', '6', 'ahmedbaracat', '8/16/2016 9:55']


['10610020', 'Ask HN: Am I the only one outraged by Twitter shutting down share counts?', '', '28', '29', 'tkfx', '11/22/2015 13:43']


['11610310', 'Ask HN: Aby recent changes to CSS that broke mobile?', '', '1', '1', 'polskibus', '5/2/2016 10:14']


['12210105', 'Ask HN: Looking for Employee #3 How do I do it?', '', '1', '3', 'sph130', '8/2/2016 14:20']


None


Below are the first five row of the show_posts list of lists:

In [ ]:
show_first_five = explore_data(show_posts,0,4)
print(show_first_five)

['10627194', 'Show HN: Wio Link  ESP8266 Based Web of Things Hardware Development Platform', 'https://iot.seeed.cc', '26', '22', 'kfihihc', '11/25/2015 14:03']


['10646440', 'Show HN: Something pointless I made', 'http://dn.ht/picklecat/', '747', '102', 'dhotson', '11/29/2015 22:46']


['11590768', 'Show HN: Shanhu.io, a programming playground powered by e8vm', 'https://shanhu.io', '1', '1', 'h8liu', '4/28/2016 18:05']


['12178806', 'Show HN: Webscope  Easy way for web developers to communicate with Clients', 'http://webscopeapp.com', '3', '3', 'fastbrick', '7/28/2016 7:11']


None


# Most Popular Posts

Now we will take our isolated lists and determine which post (Ask HN or Show HN) is more popular. To accomplish this task we will loop through each list and compute the average number of comments. The one with the highest average comments will be deemed more popular. 

First we will average the ask_hn list.

In [ ]:
total_ask_comments = 0

for row in ask_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_ask_comments += num_comments
    
avg_ask_comments = total_ask_comments / len(ask_posts) 
print(round(avg_ask_comments,0))

14.0


The average number of comments for an Ask HN post is 14. 

Lets see what we get for Show HN posts

In [ ]:
total_show_comments = 0

for row in show_posts:
    num_comments = row[4]
    num_comments = int(num_comments)
    total_show_comments += num_comments
    
avg_show_comments = total_show_comments / len(show_posts) 
print(round(avg_show_comments,0))

10.0


The average number of comments for an Show HN post is 10. 

Ask HN posts recieve 4 more comments on average than Show HN posts. Ask HN posts recieve 40% more comments from users than Show HN posts. This demonstrates that Ask HN posts recieve more engagement from our users.  If our goal is to engage our users, we should prioritize more Ask HN posts. 

Next, we will explore what times of the day get more engagment from users. 

#Most Popular Time to Post

Previously we discovered that Ask HN posts recieve 40% more comments than Show Posts. Since Ask HN posts are morely likely to receive comments, we will focus our remaing analysis on these posts. 

Next, we will determine whether certain times of the day are more likely to attract comments. We'll take the following steps to approach an answer:
1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.
2. Calculate the average number of comments ask posts receive by hour created.

In the next screen we will tackle the first step - Calculating the amount of ask posts created in each hour of the day. To accomplish this we will be importing the datetime module and creating a list called result_list that will store the time the article was created and the number of comments. 

In [ ]:
import datetime as dt

result_list = []

for row in ask_posts:
    created_at = row[6]
    num_comments = row[4]
    num_comments = int(num_comments)
    result_list.append([created_at, num_comments])

Next, we will loop through our newly created result_list and store each articles' hour as an object. 

Then we will make 2 dictionaries. The first, posts_by_hour, will store the hour as keys and the posts as values. The second, comments_by_hour will store the hour as keys and the number of comments as values.

In [ ]:
posts_by_hour = {}
comments_by_hour = {}

for row in result_list:
    hour = row[0]
    date_formatted = "%m/%d/%Y %H:%M"
    hour = dt.datetime.strptime(hour, date_formatted)
    hour = hour.strftime("%H")
    row[0] = hour
    
    if hour not in posts_by_hour:
        posts_by_hour[hour] = 1
        comments_by_hour[hour] = row[1]
    else:
        posts_by_hour[hour] += 1
        comments_by_hour[hour] += row[1]
    
print(posts_by_hour)
print("\n")
print(comments_by_hour)

{'15': 116, '18': 109, '20': 80, '07': 34, '05': 46, '23': 68, '10': 59, '17': 100, '22': 71, '19': 110, '01': 60, '09': 45, '12': 73, '21': 109, '13': 85, '11': 58, '14': 107, '04': 47, '02': 58, '00': 55, '03': 54, '08': 48, '06': 44, '16': 108}


{'15': 4477, '18': 1439, '20': 1722, '07': 267, '05': 464, '23': 543, '10': 793, '17': 1146, '22': 479, '19': 1188, '01': 683, '09': 251, '12': 687, '21': 1745, '13': 1253, '11': 641, '14': 1416, '04': 337, '02': 1381, '00': 447, '03': 421, '08': 492, '06': 397, '16': 1814}


Now that we have created our two dictionaries, we will move on to step 2 - averaging the number of comments ask posts receive by hour created.

To accomplish this we will create a new list of lists and loop over our dictionary keys and average the values.

In [ ]:
avg_by_hour = []

for hours in posts_by_hour:
    avg_by_hour.append([hours, comments_by_hour[hours]/posts_by_hour[hours]])

print(avg_by_hour)    

[['15', 38.5948275862069], ['18', 13.20183486238532], ['20', 21.525], ['07', 7.852941176470588], ['05', 10.08695652173913], ['23', 7.985294117647059], ['10', 13.440677966101696], ['17', 11.46], ['22', 6.746478873239437], ['19', 10.8], ['01', 11.383333333333333], ['09', 5.5777777777777775], ['12', 9.41095890410959], ['21', 16.009174311926607], ['13', 14.741176470588234], ['11', 11.051724137931034], ['14', 13.233644859813085], ['04', 7.170212765957447], ['02', 23.810344827586206], ['00', 8.127272727272727], ['03', 7.796296296296297], ['08', 10.25], ['06', 9.022727272727273], ['16', 16.796296296296298]]


Now that we have our list created, we need to sort it to make it readable for comparisons. 

We will start this process by swapping the average comments index from index[1] to index[0]. That way we can sort by descending average comments. 

In [ ]:
swap_avg_by_hour = []

for row in avg_by_hour:
    swap_avg_by_hour.append([row[1],row[0]])
    
print(swap_avg_by_hour)  

[[38.5948275862069, '15'], [13.20183486238532, '18'], [21.525, '20'], [7.852941176470588, '07'], [10.08695652173913, '05'], [7.985294117647059, '23'], [13.440677966101696, '10'], [11.46, '17'], [6.746478873239437, '22'], [10.8, '19'], [11.383333333333333, '01'], [5.5777777777777775, '09'], [9.41095890410959, '12'], [16.009174311926607, '21'], [14.741176470588234, '13'], [11.051724137931034, '11'], [13.233644859813085, '14'], [7.170212765957447, '04'], [23.810344827586206, '02'], [8.127272727272727, '00'], [7.796296296296297, '03'], [10.25, '08'], [9.022727272727273, '06'], [16.796296296296298, '16']]


Next we will sort by average comments in descending order using the sorted() function.

In [ ]:
sorted_swap = sorted(swap_avg_by_hour, reverse=True)

print(sorted_swap)

[[38.5948275862069, '15'], [23.810344827586206, '02'], [21.525, '20'], [16.796296296296298, '16'], [16.009174311926607, '21'], [14.741176470588234, '13'], [13.440677966101696, '10'], [13.233644859813085, '14'], [13.20183486238532, '18'], [11.46, '17'], [11.383333333333333, '01'], [11.051724137931034, '11'], [10.8, '19'], [10.25, '08'], [10.08695652173913, '05'], [9.41095890410959, '12'], [9.022727272727273, '06'], [8.127272727272727, '00'], [7.985294117647059, '23'], [7.852941176470588, '07'], [7.796296296296297, '03'], [7.170212765957447, '04'], [6.746478873239437, '22'], [5.5777777777777775, '09']]


Finally, we will display the top 5 hours for creating Ask HN posts, sorted by averaged comments. 

In [ ]:
print("Top 5 Hours for Ask Posts Comments")

from datetime import datetime, timedelta

for row in sorted_swap[:5]:
    hour = row[1]
    date_formatted = "%H"
    hour = dt.datetime.strptime(hour, date_formatted)
    hour = hour.strftime("%H:00")
    row[1] = hour
    avg_comments = row[0]
    final_string = "{}: {:.2f} average comments per post.".format(hour, avg_comments)
    print(final_string)
  

Top 5 Hours for Ask Posts Comments
15:00: 38.59 average comments per post.
02:00: 23.81 average comments per post.
20:00: 21.52 average comments per post.
16:00: 16.80 average comments per post.
21:00: 16.01 average comments per post.


# Conclusion

By far the best hour to post an Ask HN article is 3PM EST with 38.59 comments averaged for that hour. We see a 175% increase in comments from users at that time above the average of 14 comments per hour for an Ask HN post. Posting at 2AM and 8PM saw increases of 70% and 54% respectivelly. While posting at 4PM saw in increase in 20% user engagment and 9PM saw a 14% increase. 

If our goal is to increase user engagement with our website, we should post Ask HN between 3-4 PM and 8-9 PM and 2AM. In order to maximize engagement we would want to focus our efforts on posting articles at 3PM. 